<a href="https://colab.research.google.com/github/rohanguptha/INFO5731/blob/master/bompally_classwork3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class work 3: Text Classification on IMDB review samples

## Data loading

**Question 1: (10 points)**

Go to [this link](https://unt.instructure.com/courses/12025/files/2443720/download?download_frd=1) to download the data file and upload it to Google Colab. And then run the following commands:

In [0]:
import pickle

In [0]:
with open('imdb-review.pickle', 'rb') as f:
    train, test = pickle.load(f)

In [0]:
train.head()

,text,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-22-19-073406.png)

## sampling


**Question 2: (10 points)**

Sample **2000** rows from original training set. 1000 positive, 1000 negative. To keep all your result identical, please use **`random_state=1`** while sampling. Reset the index of generated data frame, and save it to **`train`**. (Hint: you can refer to[ this document](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).)






In [0]:
train_sample_positive = train[train['sentiment'] == 1].sample(n = 1000, random_state = 1)
train_sample_negitive = train[train['sentiment'] == 0].sample(n = 1000, random_state = 1)
train = train_sample_positive.append(train_sample_negitive, ignore_index=True)
train.head()

,text,sentiment
0,I have watched this episode more often than an...,1
1,"As many reviewers here have noted, the film ve...",1
2,"At first,this movie seems so bad that i almost...",1
3,Was this based on a comic-book? A video-game? ...,1
4,Melissa Joan Hart shines! This show is amazing...,1


You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-22-32-893523.png)

In [0]:
train.tail()

,text,sentiment
1995,"Is rich, ailing Elizabeth Taylor courting the ...",0
1996,The movie itself is not too bad; many comments...,0
1997,I feel much less generous with this film than ...,0
1998,It SURPRISINGLY had a plot! ;) I've seen movie...,0
1999,This movie could have been 15 minutes long if ...,0


You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-22-52-059518.png)

**Question 3: (10 points)**

Sample **1000** rows from original test set. 500 positive, 500 negative. To keep all your result identical, please use **`random_state=1`** while sampling. Reset the index of generated data frame, and save it to **`test`**. (Hint: you can refer to[ this document](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).)








In [0]:
test_sample_positive = test[test['sentiment'] == 1].sample(n = 500, random_state = 1)

test_sample_negitive = test[test['sentiment'] == 0].sample(n = 500, random_state = 1)

In [0]:
test = test_sample_positive.append(test_sample_negitive, ignore_index= True)

In [0]:
test.head()

,text,sentiment
0,Bug Juice changed my life. I Know it sounds st...,1
1,"As many of today's movies are guilty of, the p...",1
2,This is one of the best movies I have ever see...,1
3,This movie took me by surprise. The opening cr...,1
4,I am salivating for the sequels. I needed some...,1


You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-23-06-418103.png)

In [0]:
test.tail()

,text,sentiment
995,"This is my opinion of this movie, expressed in...",0
996,Why can't a movie be rated a zero? Or even a n...,0
997,The Gun is probably the worst film I've ever s...,0
998,If you are thinking of going to see this film ...,0
999,This movie was like a gathering of people that...,0


In [0]:
data = (train, test)
with open('imdb-sample.pickle', 'wb') as f:
    pickle.dump(data, f)

In [0]:
from google.colab import files
files.download('imdb-sample.pickle')

You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-23-16-486933.png)

## Classification with TextBlob

**Question 4: (30 points)**

Use [textblob](https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis) to do sentiment analysis on **test** data, and make a **`classification_report`** using the prediction result and real sentiment labels. (Hint: you can refer to [this link](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) for documentation.)








In [0]:
from textblob import TextBlob

In [0]:
pred=[]
for i in range(len(test.text)):
    test_str = test.iloc[i].text
    blob= TextBlob(test_str)
    x=blob.sentiment.polarity
    if(x<=0):
        pred.append(0);
    else:
        pred.append(1);

In [0]:
test_x = test['sentiment']

In [74]:
from sklearn.metrics import classification_report
print(classification_report(test_x,pred))

              precision    recall  f1-score   support

           0       0.91      0.43      0.59       500
           1       0.63      0.96      0.76       500

   micro avg       0.69      0.69      0.69      1000
   macro avg       0.77      0.69      0.67      1000
weighted avg       0.77      0.69      0.67      1000



You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-23-27-649327.png)

## Classification with Scikit-learn

**Question 5: (40 points)**

Train a [MultinomialNB Model](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) from Scikit-learn with the **train** data. Make sure you convert the text data into high dimensional (2000) vectors with [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), and using proper parameters to filter the tokens. Then do sentiment analysis on **test** data, and make a **`classification_report`** using the prediction result and real sentiment labels. (Hint: you can refer to [this link](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) for documentation.)








In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [70]:
vect = CountVectorizer(stop_words = 'english', token_pattern = u'(?u)\\b[^\\d\\W]\\w+\\b', max_df = 0.8, min_df = 2, max_features = 2000)
X_train = vect.fit_transform(train.text)
y_train = train.sentiment
nb = MultinomialNB()
nb.fit(X_train, y_train)
X_test = vect.transform(test.text)
y_test = test.sentiment
y_pred = nb.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       500
           1       0.84      0.78      0.81       500

   micro avg       0.81      0.81      0.81      1000
   macro avg       0.82      0.81      0.81      1000
weighted avg       0.82      0.81      0.81      1000



You should see the output like this:

![](https://github.com/wshuyi/github_pub_img/raw/master/assets/2019-03-25-21-23-45-528500.png)